In [ ]:
import sys
import os
sys.path.append('/home/581/ll6859/Documents/')
from functions import *
import matplotlib 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb
import cartopy.crs as ccrs
import seaborn as sns
from datetime import datetime

import dask
import warnings
from dask.distributed import Client

In [ ]:
plt.style.use('default')
warnings.filterwarnings('ignore')
dask.config.set(**{'array.slicing.split_large_chunks': True})
client = Client()
client

In [ ]:
# pull out location of ship
f_path = '/home/581/ll6859/Documents/CAMMPCAN/cammpcan_uw/'
voyages = sorted(gb.glob(f_path+'201819_Voyage*.csv'))
uw = pd.concat([pd.read_csv(v) for v in voyages ])
uw = uw.set_index(pd.to_datetime(uw['date_time_utc']))
uw = uw[['latitude','longitude']]
uw = uw.resample('1D', kind='Date').mean().ffill()
uw.columns = ['lat','lon']

In [ ]:
# extract chemistry data
m_path = '/g/data/jk72/slf563/ACCESS/output/cg893/daily/'
mod = 'cg893a'
dt = 'pche'
key = uw

chetrack = df_md(key, m_path, mod, dt)

In [ ]:
# convert chemistry units
chetrack = chem_unit_conversions(chetrack)

In [ ]:
# extract aerosol data
m_path = '/g/data/jk72/slf563/ACCESS/output/cg893/daily/tmp/'
mod = 'cg893a'
dt = 'paer'
key = uw

aertrack = df_md(key, m_path, mod, dt)

In [ ]:
# extract meteorology data
m_path = '/g/data/jk72/slf563/ACCESS/output/cg893/daily/'
mod = 'cg893a'
dt = 'pmet'
key = uw

mettrack = df_md(key, m_path, mod, dt)

In [ ]:
# extract u component wind data
m_path = '/g/data/jk72/slf563/ACCESS/output/cg893/daily/'
mod = 'cg893a'
dt = 'pmetu'
key = uw

metutrack = df_md(key, m_path, mod, dt)

In [ ]:
# extract v component wind data
m_path = '/g/data/jk72/slf563/ACCESS/output/cg893/daily/'
mod = 'cg893a'
dt = 'pmetv'
key = uw

metvtrack = df_md(key, m_path, mod, dt)

In [ ]:
# calculate air density from temperature
mettrack = calc_density2(mettrack)

In [ ]:
# convert aerosol units
aertrack = aero_unit_conversions2(aertrack, mettrack)

In [ ]:
# select data at 20m
aertrack = aertrack.isel(z3_hybrid_height=0, z0_hybrid_height=0).expand_dims('z0_hybrid_height').transpose()

In [ ]:
# calculate aerosol number concentrations
aertrack = nt_calcs(aertrack)

In [ ]:
# calculate CCN number concentrations
aertrack = ccn_calcs(aertrack)

In [ ]:
# long name for aerosol mode and composition fields
aertrack['field431'] = aertrack['field431'].assign_attrs({'long_name':'Dust division 1'})
aertrack['field432'] = aertrack['field432'].assign_attrs({'long_name':'Dust division 2'})
aertrack['field433'] = aertrack['field433'].assign_attrs({'long_name':'Dust division 3'})
aertrack['field434'] = aertrack['field434'].assign_attrs({'long_name':'Dust division 4'})
aertrack['field435'] = aertrack['field435'].assign_attrs({'long_name':'Dust division 5'})
aertrack['field436'] = aertrack['field436'].assign_attrs({'long_name':'Dust division 6'})

aertrack['field34101'] = aertrack['field34101'].assign_attrs({'long_name':'NUCLEATION MODE (SOLUBLE) NUMBER'})
aertrack['field34102'] = aertrack['field34102'].assign_attrs({'long_name':'NUCLEATION MODE (SOLUBLE) H2SO4'})
aertrack['field34103'] = aertrack['field34103'].assign_attrs({'long_name':'AITKEN MODE (SOLUBLE) NUMBER'})
aertrack['field34104'] = aertrack['field34104'].assign_attrs({'long_name':'AITKEN MODE (SOLUBLE) H2SO4'})
aertrack['field34105'] = aertrack['field34105'].assign_attrs({'long_name':'AITKEN MODE (SOLUBLE) BC'})
aertrack['field34106'] = aertrack['field34106'].assign_attrs({'long_name':'AITKEN MODE (SOLUBLE) OM'})
aertrack['field34107'] = aertrack['field34107'].assign_attrs({'long_name':'ACCUMULATION MODE (SOLUBLE) NUMBER'})
aertrack['field34108'] = aertrack['field34108'].assign_attrs({'long_name':'ACCUMULATION MODE (SOL) H2SO4'})
aertrack['field34109'] = aertrack['field34109'].assign_attrs({'long_name':'ACCUMULATION MODE (SOL) BC'})
aertrack['field34110'] = aertrack['field34110'].assign_attrs({'long_name':'ACCUMULATION MODE (SOL) OM'})
aertrack['field34111'] = aertrack['field34111'].assign_attrs({'long_name':'ACCUMULATION MODE (SOL) SEA SALT'})
aertrack['field34113'] = aertrack['field34113'].assign_attrs({'long_name':'COARSE MODE (SOLUBLE) NUMBER'})
aertrack['field34114'] = aertrack['field34114'].assign_attrs({'long_name':'COARSE MODE (SOLUBLE) H2SO4'})
aertrack['field34115'] = aertrack['field34115'].assign_attrs({'long_name':'COARSE MODE (SOLUBLE) BC'})
aertrack['field34116'] = aertrack['field34116'].assign_attrs({'long_name':'COARSE MODE (SOLUBLE) OM'})
aertrack['field34117'] = aertrack['field34117'].assign_attrs({'long_name':'COARSE MODE (SOLUBLE) SEA SALT'})
aertrack['field34119'] = aertrack['field34119'].assign_attrs({'long_name':'AITKEN MODE (INSOLUBLE) NUMBER'})
aertrack['field34120'] = aertrack['field34120'].assign_attrs({'long_name':'AITKEN MODE (INSOLUBLE) BC'})
aertrack['field34121'] = aertrack['field34121'].assign_attrs({'long_name':'AITKEN MODE (INSOLUBLE) OM'})
aertrack['field34126'] = aertrack['field34126'].assign_attrs({'long_name':'NUCLEATION MODE (SOLUBLE) OM'})

In [ ]:
# long name and unit changes for calculated aerosol fields
aertrack['N10'] = aertrack['N10'].assign_attrs({'units':'cm-3'})
aertrack['N10'] = aertrack['N10'].assign_attrs({'long_name':'Condensation Nuclei (>5r)'}) 

aertrack['N3'] = aertrack['N3'].assign_attrs({'units':'cm-3'})
aertrack['N3'] = aertrack['N3'].assign_attrs({'long_name':'Condensation Nuclei (>1.5r)'}) 

aertrack['CCN40'] = aertrack['CCN40'].assign_attrs({'units':'cm-3'})
aertrack['CCN40'] = aertrack['CCN40'].assign_attrs({'long_name':'Cloud Condensation Nuclei (>20r)'}) 

aertrack['CCN50'] = aertrack['CCN50'].assign_attrs({'units':'cm-3'})
aertrack['CCN50'] = aertrack['CCN50'].assign_attrs({'long_name':'Cloud Condensation Nuclei (>25r)'}) 

aertrack['CCN60'] = aertrack['CCN60'].assign_attrs({'units':'cm-3'})
aertrack['CCN60'] = aertrack['CCN60'].assign_attrs({'long_name':'Cloud Condensation Nuclei (>30r)'}) 

aertrack['DustSum'] = aertrack.field431 + aertrack.field432 + aertrack.field433 + aertrack.field434 + aertrack.field435 + aertrack.field436
aertrack['DustSum'] = aertrack['DustSum'].assign_attrs({'units':'mol/cm-3'})
aertrack['DustSum'] = aertrack['DustSum'].assign_attrs({'long_name':'Sum of dust divisions'}) 

In [ ]:
# drop height coordinate from lat and lon
aertrack['lat'] = aertrack['lat'].drop('z0_hybrid_height').squeeze()
aertrack['lon'] = aertrack['lon'].drop('z0_hybrid_height').squeeze()

In [ ]:
# calculate aerosol size distribution
aerdist = calc_size_dists(aertrack)

In [ ]:
# merges aerosol, meteorology and chemistry data together
camptrack = xr.merge([aertrack,mettrack,metutrack,metvtrack,chetrack])

In [ ]:
# drops calculated density field
camptrack = camptrack.drop('density')

In [ ]:
# drops redundant height coordinates
camptrack = camptrack.drop('z3_hybrid_height')
camptrack = camptrack.drop('z1_hybrid_height').squeeze()

In [ ]:
# adds wavelength attribute to the z_pseudo2 coordinate
camptrack['z_pseudo2'].attrs['wavelengths'] = '380 nm, 440 nm, 550 nm, 670 nm, 870 nm, 1020 nm'

In [ ]:
# remove time component from datetime coordinate, invert datetime order
camptrack['time'] = camptrack.indexes['time'].normalize()
camptrack['time'] = camptrack['time'][::-1]
aerdist['Time'] = aerdist['Time'][::-1]

In [ ]:
# add metadata attributes to netCDF

camptrack.attrs['title'] = 'ACCESS-AM2 model output along the 2018-2019 CAMMPCAN Aurora Australis voyage tracks'
camptrack.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au\nsonya.fiddes@utas.edu.au'
camptrack.attrs['description'] = 'Output from ACCESS-AM2 run for CMIP6 (research.csiro.au/access/cmip6/) with full chemistry and no ocean, nudged with ERA5, along the path of the Aurora Australis during the 2018-2019 CAMMPCAN project.'
camptrack.attrs['website'] = 'findanexpert.unimelb.edu.au/project/102792-cammpcan---chemical-and-mesoscale-mechanisms-of-polar-cell-aerosol-nucleation'

In [ ]:
# add metadata attributes to netCDF

aerdist.attrs['title'] = 'ACCESS-AM2 model aerosol size distributions along the 2018-2019 CAMMPCAN Aurora Australis voyage tracks'
aerdist.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au\nsonya.fiddes@utas.edu.au'
aerdist.attrs['description'] = 'Aerosol size distributions calculated using output from the ACCESS-AM2 run for CMIP6 (research.csiro.au/access/cmip6/) with full chemistry and no ocean, nudged with ERA5, along the path of the Aurora Australis during the 2018-2019 CAMMPCAN project.'
aerdist.attrs['website'] = 'findanexpert.unimelb.edu.au/project/102792-cammpcan---chemical-and-mesoscale-mechanisms-of-polar-cell-aerosol-nucleation'

In [ ]:
# updates netCDF history attribute
camptrack = camptrack.assign_attrs(history='/scratch/jk72/slf563/cylc-run/u-cg893/share/data/History_Data/cg893a.paer20190325 converted to netCDF by slf563 on 2022-03-30. Merged aer, chem and met into netCDF by ll6859 on 2022-05-11.')

In [ ]:
# adds netCDF history attribute
aerdist = aerdist.assign_attrs(history='Aerosol size distributions calculated with /g/data/q90/ll6859/aa1819_cg893_track.nc by ll6859 on 2022-05-11')

In [ ]:
# print size of campaign data
camptrack.nbytes

In [ ]:
# print size of aerosol distribution data
aerdist.nbytes

In [ ]:
# convert campaign data to netCDF
camptrack.load().to_netcdf(path='/g/data/q90/ll6859/aa1819_cg893_track.nc')

In [ ]:
# convert aerosol size distribution data to netCDF
aerdist.load().to_netcdf(path='/g/data/q90/ll6859/aa1819_cg893_aerdist.nc')

In [ ]:
# pull out variable names
list(camptrack.keys())

In [ ]:
# pull out variable long names
for i in list(camptrack.keys()):
    if hasattr(camptrack[i], "long_name")==True:
        print(camptrack[i].long_name)
    else:
        print(np.nan)

In [ ]:
# pull out variable coordinates
for i in list(camptrack.keys()):
    print(list(camptrack[i].coords))

In [ ]:
# pull out variable units
for i in list(camptrack.keys()):
    if hasattr(camptrack[i], "units")==True:
        print(camptrack[i].units)
    else:
        print(np.nan)